<a href="https://colab.research.google.com/github/amien1410/colab-notebooks/blob/main/RAG_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import packages
# Reinstall numpy and other packages to resolve version conflicts
!pip install numpy==1.24.4
!pip install kdbai_client langchain langchain_openai langchain-huggingface --force-reinstall --no-deps

import os

if not os.path.exists('langchain'):
    !git clone -b KDBAI_v1.4 https://github.com/KxSystems/langchain.git

if os.path.exists('langchain/libs/community'):
    os.chdir('langchain/libs/community')
    !pip install . --force-reinstall --no-deps
else:
    print("Error: 'langchain/libs/community' directory not found. Git clone might have failed.")

  Using cached kdbai_client-1.6.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_openai-0.3.19-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_huggingface-0.2.0-py3-none-any.whl.metadata (941 bytes)
Using cached kdbai_client-1.6.1-py3-none-any.whl (34 kB)
Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
Using cached langchain_openai-0.3.19-py3-none-any.whl (64 kB)
Using cached langchain_huggingface-0.2.0-py3-none-any.whl (27 kB)
  Attempting uninstall: langchain_openai
    Found existing installation: langchain-openai 0.3.19
    Uninstalling langchain-openai-0.3.19:
      Successfully uninstalled langchain-openai-0.3.19
  Attempting uninstall: langchain-huggingface
    Found existing installation: langchain-huggingface 0.2.0
    Uninstalling langchain-huggingface-0.2.0:
      Successfully uninstalled langchain-huggingface-0.2.0
  Attempting uninstall: kdbai_client
    Found existing insta

In [2]:
# download State of the Union speech data
import os

if os.path.exists("./data/state_of_the_union.txt") == False:
    !mkdir ./data
    !wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/retrieval_augmented_generation/data/state_of_the_union.txt

In [3]:
# vector DB
from getpass import getpass
import kdbai_client as kdbai
import time

# langchain packages
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import KDBAI
from langchain import HuggingFaceHub
from langchain_openai import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = (
    os.environ["HUGGINGFACEHUB_API_TOKEN"]
    if "HUGGINGFACEHUB_API_TOKEN" in os.environ
    else getpass("Hugging Face API Token: ")
)

In [ ]:
# Load the documents we want to prompt an LLM about
doc = TextLoader("data/state_of_the_union.txt").load()

In [6]:
# Chunk the documents into 500 character chunks using langchain's text splitter "RucursiveCharacterTextSplitter"
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

# split_documents produces a list of all the chunks created, printing out first chunk for example
pages = [p.page_content for p in text_splitter.split_documents(doc)]

pages[0]

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.'